#### TMA4205 Numerical Linear Algebra
# Project Part 2
**Author:** Alexander Johan Arntzen

**Date:** 22.11.2021
## Introduction

In this part of the project we want to find a low rank approximation to a matrix valued function. More precisely let $\cal{M_k}$ denote the manifold of $m \times n$ matrices of rank $k$. For all $t \in I$ we want to find $X(t) \in \cal{M_k}$ such that

\begin{equation}
    ||X(t) - A(t)||_F
\end{equation}

is minimized. For each $k$ we already know that truncated singular value decomposition is a solution the above problem. $X(t)$ computed by SVD is however expensive to compute and can be discontinuous. Therefore, we consider two alternative ways of computing a low-rank approximation to $A(t)$:  Lanczos bidiagonalization and dynamic low-rank approximation.

In [ ]:
"""This cell is just for customize the notebook experinece. No math here please move on.."""
%load_ext autoreload
%autoreload 2

# imports and useful functions
from IPython.display import clear_output
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats

set_matplotlib_formats('pdf', 'svg')

# line cyclers adapted to colourblind people
from cycler import cycler

line_cycler = (cycler(color=["#E69F00", "#56B4E9", "#009E73", "#0072B2", "#D55E00", "#CC79A7", "#F0E442"]) +
               cycler(linestyle=["-", "--", "-.", ":", "-", "--", "-."]))
plt.rc("axes", prop_cycle=line_cycler)
plt.rc('axes', axisbelow=True)

In [ ]:
"""Define some useful functions for plotting and testing"""
from linalg.helpers import get_function_timings, truncated_svd, get_equidistant_indexes, get_best_approx

## Exercise 1:
### Lanczos bidiagonalization
When approximating with Lanczos bidiagonalization we stop Lanczos algorithm [2] after $k$ steps. This yields a decomposition of $A$ to the form $Q_k B_k P_k^T$. where $B_k$ is a $k \times k$ lower bidiagonal matrix and $Q_k \in \mathbb{R}^{m \times k}$, $P_k \in \mathbb{R}^{n \times k}$ have orthonormal columns.

One problem with lanczos algorithm is that it suffers loss of orthogonality of the columns of $P$ and $Q$. In [5] it is shown that performing re-orthogonalization with the columns of $Q$ preserves the orthogonality of both $P$ and $Q$.


In the code block below Lanczos bidiagonalization method with and without re-orthogonalization is implemented.

In [ ]:
"""Implement Lanczos bidiagonalization"""
from linalg.bidiagonalization import lanczos_bidiag, lanczos_bidiag_reorth, make_bidiagonal, get_bidiagonal_approx

### Test Lanczos bidiagonalization

To test Lanczos bidiagonalization we randomly generate 3 matrices $A_n \in \mathbb{R}^{n\times n} $, $n \in [32,64,128]$.
For each $n$ the singular values of $A_n$ are shown below.

In [ ]:
"""Print singular values of A_n"""
# make matrices
n_list = [32, 64, 128]
A_list = [np.random.rand(n, n) * 2 - 1 for n in n_list]

# plot their eigenvalues
fig, axs = plt.subplots(ncols=len(n_list), sharey=True, constrained_layout=True, figsize=(3 * len(n_list) + 1, 4))
fig.suptitle("Singular values of $A_n$")
for A, ax, n in zip(A_list, axs, n_list):
    svs = np.linalg.svd(A, compute_uv=False)
    ax.plot(svs, ".")
    ax.set_ylabel("$\sigma$")
    ax.set_xlabel("$\sigma$ number")
    ax.set_title(f"$n={n}$")

Now we approximate the matrices $A_n$ with truncated SVD $X(t)$ and Lanczos bidiagonalization $W(t)$ for all $k \leq n$.
The approximation error in Frobenius norm is then shown in the figure below. Note that truncated SVD gives the best approximation of rank $k$ (in Frobenius norm).

Since Lanczos bidiagonalization can be computed with and without reorthogonalization approximation, both methods are included in the experiment.

We also measure the orthogonality error with the method from [5]:

\begin{equation}
    \eta(U) :=  ||I -U^T U||_F
\end{equation}

Lanczos bidiagonalization decomposes a matrix into $ W = PBQ^T$ where $P$ and $Q$ are orthonormal. For each $k$ we show the orthogonality error $\eta(P)$ and $\eta(Q)$ for bidiagonalization with and without re-orthogonalization.

In [ ]:
# Make figure for showing approximation error
fig, axs = plt.subplots(ncols=len(n_list), tight_layout=True, figsize=(1 + 3 * len(n_list), 4))
axs[0].set_ylabel("$||A - A_k||_F$")
fig.suptitle("Approximation error with methods of rank $k$ ")

# Make figure for showing orthogonality error
orth_fig, orth_axs = plt.subplots(ncols=len(n_list), sharey=True, tight_layout=True, figsize=(1 + 3 * len(n_list), 4))
orth_fig.suptitle("Orthogonality error for bidiagonalization of rank $k$")
orth_axs[0].set_ylabel("$\eta$")
for A, n, ax, orth_ax in zip(A_list, n_list, axs, orth_axs):
    # initialize
    best_approx_error = np.zeros(n)
    bidiagonal_error = np.zeros(n)
    bidiagonal_reorth_error = np.zeros(n)

    # error: [P, Q, P_reorth, Q_reorth]
    reorth_error = np.zeros((n, 4))

    k_list = np.arange(1, n + 1)
    for i, k in enumerate(range(1, n + 1)):
        b = np.random.rand(n)
        # compute bidiagonalization
        P, Q, alpha, beta = lanczos_bidiag(A, k, b)
        B = make_bidiagonal(alpha, beta)
        bidiagonal_error[i] = np.linalg.norm(A - P @ B @ Q.T, ord="fro")
        reorth_error[i, 0] = np.linalg.norm(np.eye(k) - P.T @ P, ord="fro")
        reorth_error[i, 1] = np.linalg.norm(np.eye(k) - Q.T @ Q, ord="fro")

        # compute bidiagonalization with reroth
        P, Q, alpha, beta = lanczos_bidiag_reorth(A, k, b)
        B = make_bidiagonal(alpha, beta)
        bidiagonal_reorth_error[i] = np.linalg.norm(A - P @ B @ Q.T, ord="fro")
        reorth_error[i, 2] = np.linalg.norm(np.eye(k) - P.T @ P, ord="fro")
        reorth_error[i, 3] = np.linalg.norm(np.eye(k) - Q.T @ Q, ord="fro")

        # compare with best approximation of rank k
        A_k = get_best_approx(A, k)
        best_approx_error[i] = np.linalg.norm(A - A_k, ord="fro")

    # plot approximation error
    ax.plot(k_list, best_approx_error, label="best approximation")
    ax.plot(k_list, bidiagonal_error, label="bidiagonalization")
    ax.plot(k_list, bidiagonal_reorth_error, label="bidiag. with reorth.")
    ax.set_xlabel("$k$")
    ax.set_title(f"$n={n}$")
    ax.legend()

    #plot orthogonality error
    orth_ax.semilogy(k_list, reorth_error[:, 0], label="P")
    orth_ax.semilogy(k_list, reorth_error[:, 1], label="Q")
    orth_ax.semilogy(k_list, reorth_error[:, 2], label="P with reorth.")
    orth_ax.semilogy(k_list, reorth_error[:, 3], label="Q with reorth.")
    orth_ax.set_xlabel("$k$")
    orth_ax.set_title(f"$n={n}$")
    orth_ax.legend()
plt.show()

From the figures above we see that bidiagonalization without re-orthogonalization does not give good approximations for large $k$.
The reason for this is that numerical instability makes the columns of $Q$ and $P$ not orthonormal.

## Exercise 2
### Dynamic low-rank approximation
Assume that the matrix valued function $A(t)$ is differentiable. The dynamical low-rank approximation $Y$  of $A$ is the curve satisfying

\begin{equation}
 \dot Y(t) = \underset{V \in T_{Y(t)}\cal{M_k}}{\text{argmin}} ||V - \dot A(t)||_F.
\end{equation}

In other words $\dot Y(t)$ is the orthogonal projection of $\dot A(t)$ onto $T_{Y(t)}\cal{M_k}$ in the Frobenius inner product. Thus computing $Y$ corresponds to solving an initial value problem.

Furthermore, it can be shown [4] that if Y has decomposition $Y = USV^T$ with $U,V$ orthonormal and  $U^T\dot U = V^T\dot V = 0$. Then the derivative $\dot Y$ is given by.

\begin{align}
    \dot Y &= \dot USV^T + U\dot SV^T + US\dot V^T \\
    \text{where } \\
    \dot S &= U^T\dot AV\\
    \dot U &= (I - UU^T)\dot A V S^{-1}\\
    \dot V &= (I - VV^T)\dot A^T U S^{-T}\\
\end{align}
### Time integration
To solve the initial value problem we use two Runge-Kutta schemes. One fist order scheme to estimate the time step and one second order scheme that takes the actual step. Thus, the difference between these steps gives an estimate on the local truncation error. An adaptive step size-strategy is implemented by only accepting steps when the estimate for the local truncation error is below a tolerance, otherwise reducing the step size. For further details we refer to the project description.

To preserve the orthogonality of $U$ and $V$ the Caylay map is used. The Caylay map is defined by
\begin{align}
\text{cay}(B) :&= \big(I - \frac{1}{2}B\big)^{-1}\big(I + \frac{1}{2}B\big).
\end{align}

Furthermore, if $B$ is a skew symmetric matrix then it can be shown that $\text{cay}(B)$ is orthonormal.

To apply the Caylay map, the ODE for $Y$ is rewritten so that we have

\begin{aligned}
    \dot U &= G_U(Y)U \\
    \dot V &= G_V(Y)V,
\end{aligned}

where $G_U(Y)$ and  $G_V(Y)$ are skew symmetric. Thus, we can preserve the orthogonality of the columns of $U$ and $V$ by taking timesteps

\begin{aligned}
    U_{i+1} &= \text{cay}(hG_U(Y_i))U_i \\
    U_{i+1} &= \text{cay}(hG_U(Y_i))U_i. \\
\end{aligned}

This leads to a first order Runge-Kutta scheme:

\begin{align}
S_{j+1} &= S_j + hU_j^T\dot A_j V_j\\
U_{j+1} &= \text{cay}\big(h(F_{U_j}U_j^T - U_jF_{U_j}^T) \big)U_j \\
V_{j+1} &= \text{cay}\big(h(F_{V_j}V_j^T - V_jF_{V_j}^T) \big)V_j \\
\\
\text{where } \\
F_{U_j} :&= (I - U_jU_j^T)\dot A_j V_j S_j^{-1} \\
F_{V_j} :&= (I - V_jV_j^T)\dot A_j^T U_j S_j^{-T}. \\
\end{align}

For the second order Runge-Kutta scheme we refer to the project description.

In [ ]:
from linalg.integrate import *

### Efficient computation of the caylay map

To make sure that the resulting matrices are in fact orthogonal we take steps in using the caylay-map.
Furthermore, all inputs in they caylay map are given on the form $B =C D^T = [F, -U] [U, F]^T$.
Where $U^TU = I$ and $F^T U=0$.
This input form we exploit to compute the caylay map more efficiently. In the problem description it is proven that for input $B = C D^T$

\begin{equation}
\text{cay}(CD^T) = I + C(I-\frac{1}{2}D^TC)^{-1}D^T
\end{equation}

This reduces the computational cost of calculating the inverse matrix from $O(m^3)$ to $O(k^3)$.

Using the form above and the fact and that $C D^T = [F, -U] [U, F]^T$, we get the following formula for the Caylay map

\begin{align}
    \text{cay}(CD^T) = I + C(
    \begin{bmatrix}
      A&-0.5A\\
      0.5QA& A
    \end{bmatrix})D^T,
\end{align}

where

\begin{equation}
   A:= (I-  \frac{1}{4}F^T F)^{-1}.
\end{equation}

This might reduce the computational complexity since $A$ is half the size of $(I-\frac{1}{2}D^TC)$.

The three different implementations of the Caylay map are implemented below.

In [ ]:
"""Implement Caylay maps"""
from linalg.cayley_map import cayley_map_simple, cayley_map_efficient_mod, cayley_map_efficient

### Test function
In ths exercise we consider $k$-rank approximations of the solution to the matrix equation:

\begin{equation}
\dot A(t) =BA(t), A(0) = A_0,
\end{equation}

where $B$ is a linear operator $B: \mathbb{R}^{m \times n} \rightarrow \mathbb{R}^{m \times n}$.
This equation will have solution $A(t) = \exp({tB})A_0$.

In our test problem we let $B$ be the discrete laplacian operator $L_h$.
Thus, $A(t)$ will be the time evolution of the space discretized heat equation in two space dimensions;
with $m,n$ nodes in the $x, y$ directions respectively.
Computing the matrix exponential will be an expensive operation with cost $ O((nm)^{3}$). Therefore, we transform the original ODE to a ODE with only matrix products.

For any time $t$ a discretized heat matrix can be considered as the sum of outer products

\begin{equation}
A(t) = \sum_{i = 1}^{\min(m,n)} u_i \otimes v_i,
\end{equation}

where $u_i$ and $v_i$ are a vectors varying in the $x$ and $y$ directions respectively.
Since the discrete laplacian is the sum of the double derivative in each direction it can be decomposed as:

\begin{equation}
L_h(A(t))= \sum_{i = 1}^{\min(m,n)} D_{xx} u_i \otimes v_i +\sum_{i  = 1}^{\min(m,n)} u_i \otimes D_{yy} v_i = D_{xx}A(t) + A(t)D_{yy}^T.
\end{equation}

Here $D_{xx}$ and $D_{yy}$ are the discrete laplacians in one dimension.
Thus, the solution to the matrix ODE with $B = L_h$ is:

\begin{equation}
A(t)= \exp(tD_{xx})A_0 \exp(tD_{yy}^T).
\end{equation}

We now only have to compute these two exponentials with cost $O(m^3+ n^3)$.

Finally, to ensure that $A(t)$ has rank $k$ we define $ A_0 := C_0 D_0^T $, where $C_0 \in \mathbb{R}^{m \times k}$ and $D_0 \in \mathbb{R}^{n \times k}$ are randomly generated.

Methods for computing $A_0, \ A $ and $\dot A$ are implemented in the code cell below.

In [ ]:
from test.case_matrix_ode import generate_heat_equation

### Testing the caylay map with the heat equations
We now test the three different implementations of the Caylay map on the discriteized heat equation at $t=0$. That is we compute the Caylay map of $B =C D^T = [F_U, -U] [U, F_U]^T$, where $U, F_U$ are as given in the first order Runge-Kutta scheme in the first time step. The test where performed with $k = \sqrt{m}$ and $m \in [16,2048]$.

In [ ]:
"""Implement test case for """
from test.test_caylay import get_FUCDB

# compute the matrices to test the Caylay map on
m_list = 2 ** np.array([4, 5, 6, 7, 8, 9, 10, 11])
sample_FUCDB = [get_FUCDB(m, k=int(np.sqrt(m))) for m in m_list]
sample_FU = [(F, U) for F, U, C, D, B in sample_FUCDB]
sample_CD = [(C, D) for F, U, C, D, B in sample_FUCDB]
sample_B = [(B,) for F, U, C, D, B in sample_FUCDB]

# time computation
time_simple = get_function_timings(cayley_map_simple, sample_B, number=10)
time_efficient = get_function_timings(cayley_map_efficient, sample_CD, number=10)
time_mod = get_function_timings(cayley_map_efficient_mod, sample_FU, number=10)

# plot results
fig, ax = plt.subplots()
fig.suptitle("Performance of different methods of caylay map computation")
ax.loglog(m_list, time_simple, label="simple", base=2)
ax.loglog(m_list, time_efficient, label="efficient", base=2)
ax.loglog(m_list, time_mod, label="efficient modified", base=2)
ax.set_ylabel("Time [ms]")
ax.set_xlabel("$m$")
ax.legend()
plt.show()

We see that the two modified Caylay map implementations are the most efficient implementations for our case.

### Testing the dynamic low-rank approximation with the discretized heat equation
We now test the dynamic low-rank approximation on the solution to the discretized heat equation defined above. We use truncated SVD of $A_0$ as initial value when integrating the ODE. The approximation error of the dynamical low-rank approximation $Y$, truncated SVD $X(t)$ and time derivative $\dot Y(t)$ of $Y(t)$ is then computed.

In [ ]:
"""Test different approximation methods on the discretized heat equation"""
m = 30
t_f = 1
k_list = [10, 20]

# make figures to plot errors
fig, axs = plt.subplots(ncols=len(k_list), nrows=2, sharex=True, constrained_layout=True,
                        figsize=(1 + 3 * len(k_list), 4 + 3))
axs[0, 0].set_ylabel("Frobenious norm")
axs[1, 0].set_ylabel("Frobenious norm")
fig.suptitle("Error for different low rank approximations")

for i, k in enumerate(k_list):
    print(f"Running k={k}")
    # generate case and start conditions
    A_0, A, A_dot = generate_heat_equation(n=m, m=m, k=k)
    Y_0 = truncated_svd(A_0, k)

    # integrate
    Y, T = matrix_ode_simple(0, t_f, Y_0=Y_0, X=A_dot, TOL=1e-4, h_0=1e-8, verbose=True)
    t_ind = get_equidistant_indexes(T, 0, t_f)
    T = [T[i] for i in t_ind]
    Y = [Y[i] for i in t_ind]

    # calculate errors in fro norm using previously defined functions
    XA_diff = [np.linalg.norm(get_best_approx(A(t), k) - A(t), ord="fro") for t in T]
    YA_diff = [np.linalg.norm(multiply_factorized(*y) - A(t), ord="fro") for t, y in zip(T, Y)]
    YA_dot_diff = [np.linalg.norm(get_y_dot(A_dot=A_dot, Y=y, t=t) - A_dot(t), ord="fro") for t, y in zip(T, Y)]
    YX_diff = [np.linalg.norm(multiply_factorized(*y) - get_best_approx(A(t), k), ord="fro") for t, y in zip(T, Y)]

    # plot errors
    ax_u = axs[0, i]
    ax_u.set_title(f"$k={k}$")
    ax_u.plot(T, YX_diff, label="||Y - X||")
    ax_u.plot(T, YA_diff, label="||Y - A||")
    ax_u.plot(T, YA_dot_diff, label="$||\dot{Y} - \dot{A}||$")
    ax_u.legend()

    ax_l = axs[1, i]
    ax_l.plot(T, XA_diff, label="||X - A||")
    ax_l.set_xlabel("$t$")
    ax_l.legend()
    clear_output()
plt.show()

The approximation error of truncated SVD $X(t)$ is low. This was expected since $A(t)$ has rank $k$ and truncated SVD will approximate rank $k$ matrices perfectly. The remaining error is probably due to limitations of floating point operations.

We also se that the difference $\dot Y- \dot A$ is largest for the first time steps. This can be explained by the randomly generated matrix $A_0$ making the initial derivatives large. That is, the large differences between entries of $A_0$ makes $\dot A = L_h(A_t)$ large. Thus, a larger error in $||\dot Y(t)- \dot A(t)||_F$ is expected. This gives an initial error in the dynamic low-rank approximation which is persistent for later times. Restarting the integration of the dynamic low-rank approximation should mitigate this type of error.

## Exercise 3
We now consider the first example in section 3 of [4]. Here the matrix to be approximated is defined as

\begin{equation}
ÅA(t) = Q_1(t)\bigg(A_1 + e^{t}A_2\bigg)Q_2^T(t).
\end{equation}

Here $Q_i$ are orthonormal matrices given by $\dot Q_i= T_i Q_i, \ i=1,2$,
where $T_i$ are skew symmetric matrices with constant diagonals.
$A_1$ and $A_2$ are randomly generated matrices in $[0,5]^{100 \times 100}$ with 10 singular values ≈ 1.
$\epsilon$ is a parameter regulating how much noise is added to the $A_1$ and $A_2$ matrices.
See [4] for further details.

The test problem is implemented in the cell below.

In [ ]:
"""Implement first example """
from test.case_matrix_ode import generate_first_example

Using the test function defined above, we test the following approximation methods:
* Truncated SVD: $X$ (the best approximation of rank $k$)
* Lanczos bidiagonalization method: $W$
* Dynamic low-rank approximation: $Y$

Each approximation method of rank $k \in \{10,20\}$ was tested with $\epsilon \in \{1e-1, 1e-2, 1e-3, 1e-4, 1e-5\}$.

In [ ]:
t_f = 1
eps_list = 10. ** np.array([-1, -2, -3, -4, -5])
k_list = [10, 20]
m = 100

# make figures for plotting errors
fig, axs = plt.subplots(nrows=len(eps_list), ncols=len(k_list), sharex=True, squeeze=False, constrained_layout=True,
                        figsize=(1 + 3 * len(k_list), 1 + 3 * len(eps_list)))
fig.suptitle("Error for different low rank approximations")

# iterate over parameters
for i, eps in enumerate(eps_list):
    axs[i, 0].set_ylabel("Frobenious norm")
    for j, k in enumerate(k_list):
        axs[-1, j].set_xlabel("$t$")
        print(f"k: {k}, epsilon: {eps}")

        # generate case and start conditions
        A_0, A, A_dot = generate_first_example(eps=eps)
        Y_0 = truncated_svd(A_0, k)

        # integrate
        Y, T = matrix_ode_simple(0, t_f, Y_0=Y_0, X=A_dot, TOL=1e-3, verbose=True)
        t_ind = get_equidistant_indexes(T, 0, t_f, n=m)
        T = [T[i] for i in t_ind]
        Y = [Y[i] for i in t_ind]

        # calculate errors in fro norm using previously defined functions
        b = np.random.rand(m)
        XA_diff = [np.linalg.norm(get_best_approx(A(t), k) - A(t), ord="fro") for t in T]
        YA_diff = [np.linalg.norm(multiply_factorized(*y) - A(t), ord="fro") for t, y in zip(T, Y)]
        YA_dot_diff = [np.linalg.norm(get_y_dot(A_dot=A_dot, Y=y, t=t) - A_dot(t), ord="fro") for t, y in zip(T, Y)]
        WA_diff = [np.linalg.norm(get_bidiagonal_approx(A(t), k=k, b=b) - A(t), ord="fro") for t in T]
        YX_diff = [np.linalg.norm(multiply_factorized(*y) - get_best_approx(A(t), k), ord="fro") for t, y in zip(T, Y)]

        # plot errors
        ax = axs[i, j]
        ax.set_title(f"$k=${k}, $\epsilon =$ {eps}")
        ax.plot(T, XA_diff, label="$||X - A||$")
        ax.plot(T, YA_diff, label="$||Y - A||$")
        ax.plot(T, YA_dot_diff, label="$||\dot{Y} - \dot{A}||$")
        ax.plot(T, WA_diff, label="$||W - A||$")
        ax.plot(T, YX_diff, label="$||Y - X||$")
        ax.legend()
        clear_output()
plt.show()

From the experiments above we can make 3 main observations.
Firstly, the dynamic low-rank approximation $Y(t)$ approximates $A(t)$ well compared to the best approximation $X(t)$.

Secondly, the error $||X-A||_F $ is more dependent of the on the  noise level $\epsilon$ added to $A_1$ and $A_2$ than the rank $k$.
The reason for this is that $A(t)$ has 10 dominant singular values which are captured by $Y(t)$ when $k\geq10$.
Increasing $\epsilon$ also increases the remaining singular values.
Increasing $k$ will make the approximation more accurate, but with increasing $\epsilon$ the remaining singular values still grow larger.

Finally, as noted in the previous exercise, Lanczos bidiagonalization $W(t)$ needs higher rank to approximate a matrix as well as the best approximation $X(t)$.
Therefore, it is not strange that Lanczos bidiagonalization of rank $k=10$ would not capture all the 10 dominant singular values of $A(t)$.
For $k=10$, $W(t)$ therefore shows a large error relative to $X(t)$.
However, for $k=20$ the difference between $X(t)$ and $W(t)$ is not as large.
This indicates that $W(t)$ with $k=20$ manages to capture the 10 dominant singular values.

## Exercise 4
We now implement the second example in section 3 of [4]. Here $A(t)$ is defined as

\begin{equation}
A(t) = Q_1(t)\bigg(A_1 + \cos(t)A_2\bigg)Q_2^T(t),
\end{equation}

where $Q_i$ and $A_i, \ i={1,2}$ are defined as in the previous example.
Notice, however, that $e^t$ has been replaced by $\cos(t)$.

The example is implemented in the code block below.

In [ ]:
"""Implement second example """
from test.case_matrix_ode import generate_second_example

We now test the the dynamical low-rank approximation $Y(t)$ and truncated SVD $X(t)$ of rank $k=\{5,20\}$ on the test problem with $\epsilon = 0.1$.
We also find the singular values of each approximation method.

In [ ]:
t_f = 10
eps_list = [1e-1]
k_list = [5, 20]
m = 100
# define figures for plotting singular values
fig_sigma, axs_sigma = plt.subplots(nrows=len(eps_list), ncols=len(k_list), sharex=True, sharey=True, squeeze=False,
                                    constrained_layout=True,
                                    figsize=(1 + 3 * len(k_list), 1 + 3 * len(eps_list)))
fig_sigma.suptitle("Singular values over time")

# define figures for plotting approximaton error
fig, axs = plt.subplots(nrows=len(eps_list), ncols=len(k_list), sharex=True, squeeze=False, constrained_layout=True,
                        figsize=(1 + 3 * len(k_list), 1 + 3 * len(eps_list)))
fig.suptitle("Error for different low rank approximations")

# iterate over parameters
for i, eps in enumerate(eps_list):
    axs[i, 0].set_ylabel("Frobenious norm")
    for j, k in enumerate(k_list):
        axs[-1, j].set_xlabel("$t$")
        axs_sigma[-1, j].set_xlabel("$t$")
        print(f"k: {k}, epsilon: {eps}")

        # generate case and start conditions
        A_0, A, A_dot = generate_second_example(eps=eps)
        Y_0 = truncated_svd(A_0, k)

        # integrate
        Y, T = matrix_ode_simple(0, t_f, Y_0=Y_0, X=A_dot, TOL=1e-3, verbose=True)

        # store a subset instead
        t_ind = get_equidistant_indexes(T, 0, t_f, n=2 * m)
        T = [T[i] for i in t_ind]
        Y = [Y[i] for i in t_ind]

        # calculate errors in fro norm for methods already defined
        XA_diff = [np.linalg.norm(get_best_approx(A(t), k) - A(t), ord="fro") for t in T]
        YA_diff = [np.linalg.norm(multiply_factorized(*y) - A(t), ord="fro") for t, y in zip(T, Y)]
        YA_dot_diff = [np.linalg.norm(get_y_dot(A_dot=A_dot, Y=y, t=t) - A_dot(t), ord="fro") for t, y in zip(T, Y)]
        YX_diff = [np.linalg.norm(multiply_factorized(*y) - get_best_approx(A(t), k), ord="fro") for t, y in zip(T, Y)]
        A_norm = np.array([np.linalg.norm(A(t), ord="fro") for t in T])


        # plot approximation errors
        ax = axs[i, j]
        ax.set_title(f"$k=${k}, $\epsilon =$ {eps}")
        ax.plot(T, XA_diff, label="||X - A||")
        ax.plot(T, YA_diff, label="||Y - A||")
        ax.plot(T, YA_dot_diff, label="$||\dot{Y} - \dot{A}||$")
        ax.plot(T, YX_diff, label="||Y - X||")
        ax.legend()

        # calculate singular values of X and Y
        sing_values = np.linalg.svd([A(t) for t in T], compute_uv=False)
        sing_values_y = np.linalg.svd([S for U, S, V in Y], compute_uv=False)

        # plot singular values
        ax_sigma = axs_sigma[i, j]
        ax_sigma.set_ylabel("$\sigma_i$")
        ax_sigma.set_title(f"$k=${k}, $\epsilon =$ {eps}")
        l0, *_ = ax_sigma.plot(T, sing_values, "k-", lw=0.5, label="A(t)")
        l1, *_ = ax_sigma.plot(T[::4], sing_values_y[::4, :k], "r.", alpha=0.5, label="Y(t)")
        ax_sigma.legend(handles=[l0, l1])

        clear_output()
plt.show()


This example illustrates how the dynamical low rank approximation $Y(t)$ as no error bounds in general.
The authors of [4] give error bound with under regularity conditions for the best-approximation $X(t)$.
In this case however, the rank $k=5$ best-approximation $X(t)$ is discontinuous, and thus we have no error bounds.

More specifically the low rank approximation fails because the singular values of $A(t)$ cross multiple times.
To see how this affects the low rank approximation we consider an initial ordering of the singular values of $A(0)$ as $\sigma_i, 1 \leq i \leq 100$.
Now since $A$ has the form

\begin{equation}
A(t) = Q_1(t)\bigg(A_1 + \cos(t)A_2\bigg)Q_2^T(t),
\end{equation}

we can write $A(t)$ as sum of smooth functions

\begin{equation}
A(t) = \sum_{i=1}^{100} u_i(t)v_i(t)^T \sigma_i (t).
\end{equation}

Now let $t_k$ be the first time when $\sigma_{j}(t) \leq \sigma_{i}(t), j \leq k < i $.
Then for $0 \leq t<t_k$:

\begin{equation}
X(t) = \sum_{i=1}^{k} u_i(t)v_i(t)^T \sigma_i (t),
\end{equation}

but at $t_k$ $X$ will change functions with index $i$ for index $j$ and have a discontinuity. We know it has to be a discontinuity because all $u_i$'s and $v_i$'s are smooth functions pointwise orthogonal to each other.
On the other hand, the dynamical low rank approximation is smooth and will thus approximate the $k$ singular values that where largest,
even though they are not largest anymore.

This crossing singular values can be seen in figure above.
Here we see that when singular values not approximated by $Y(t)$ becomes large.
Then the approximated singular stops to follow the curve of singular values of $A(t)$.
Also in the figure below the errors of different approximations are shown.
We see that when singular values cross, then the difference between the best approximation $X$ and the dynamical low rank approximation  $Y$ grows larger.
This effect is not as large for trials with $k=20$ since $Y(t)$ can capture more singular values. This type of error could be avoided by restarting the integration as is in [4].

## Bibliography

[1] G.H. Golub and C.F. Van Loan, *Matrix Computations*, Johns Hopkins Studies in the Mathematical Sciences.

[2] G.H. Golub and W. Kahan, *Calculating the singular values and pseudo-inverse of a
matrix*, SIAM J. Numer. Anal., 2 (1965), pp 205–224.

[3] G.H. Golub and C.F. Van Loan, Matrix Computations, Johns Hopkins Studies in the
Mathematical Sciences.

[4] O. Koch and C.Lubic *Dynamical low-rank approximation* SIAM J. on Matrix Anal. and Appl. (2007), DOI 10.1137/050639703.

[5] H.D. Simon and H. Zha, *Low-rank matrix approximation using the Lanczos bidiagonalization process with applications*, SIAM J. Sci. Comp., vol 6 (2000), pp 2257–2274.

